In the previous session we trained a model for predicting churn and evaluated it. Now let's deploy it

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [6]:
# telco customer churn prediction
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'
!curl $data -o data-week-3.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  954k  100  954k    0     0  2949k      0 --:--:-- --:--:-- --:--:-- 2946k


In [8]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [11]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [12]:
# shape data
df_full_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5634 entries, 1814 to 5157
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        5634 non-null   object 
 1   gender            5634 non-null   object 
 2   seniorcitizen     5634 non-null   int64  
 3   partner           5634 non-null   object 
 4   dependents        5634 non-null   object 
 5   tenure            5634 non-null   int64  
 6   phoneservice      5634 non-null   object 
 7   multiplelines     5634 non-null   object 
 8   internetservice   5634 non-null   object 
 9   onlinesecurity    5634 non-null   object 
 10  onlinebackup      5634 non-null   object 
 11  deviceprotection  5634 non-null   object 
 12  techsupport       5634 non-null   object 
 13  streamingtv       5634 non-null   object 
 14  streamingmovies   5634 non-null   object 
 15  contract          5634 non-null   object 
 16  paperlessbilling  5634 non-null   object 
 1

In [13]:
df_full_train.head()

customerid  gender  seniorcitizen partner dependents  tenure  \
1814  5442-pptjy    male              0     yes        yes      12   
5946  6261-rcvns  female              0      no         no      42   
3881  2176-osjuv    male              0     yes         no      71   
2389  6161-erdgd    male              0     yes        yes      71   
3676  2364-ufrom    male              0      no         no      30   

     phoneservice multiplelines internetservice       onlinesecurity  ...  \
1814          yes            no              no  no_internet_service  ...   
5946          yes            no             dsl                  yes  ...   
3881          yes           yes             dsl                  yes  ...   
2389          yes           yes             dsl                  yes  ...   
3676          yes            no             dsl                  yes  ...   

         deviceprotection          techsupport          streamingtv  \
1814  no_internet_service  no_internet_service  no_internet_service   
5946                  yes                  yes                   no   
3881                   no                  yes                   no   
2389                  yes                  yes                  yes   
3676                   no                  yes                  yes   

          streamingmovies  contract paperlessbilling  \
1814  no_internet_service  two_year               no   
5946                  yes  one_year               no   
3881                   no  two_year               no   
2389                  yes  one_year               no   
3676                   no  one_year               no   

                  paymentmethod monthlycharges  totalcharges  churn  
1814               mailed_check          19.70        258.35      0  
5946    credit_card_(automatic)          73.90       3160.55      1  
3881  bank_transfer_(automatic)          65.15       4681.75      0  
2389           electronic_check          85.45       6300.85      0  
3676           electronic_check          70.40       2044.75      0  

[5 rows x 21 columns]

In [14]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [15]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [16]:
C = 1.0
n_splits = 5

In [17]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

/Users/mfilipav/dev/ml-scripts/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mfilipav/dev/ml-scripts/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

C=1.0 0.842 +- 0.007


/Users/mfilipav/dev/ml-scripts/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
scores

[np.float64(0.8444866591831448),
 np.float64(0.8451136438882267),
 np.float64(0.8332851695765604),
 np.float64(0.834684961949223),
 np.float64(0.851812776505287)]

In [19]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.8583842503818778)

## Save the model

In [20]:
import pickle

In [21]:
output_file = f'model_C={C}.bin'

In [22]:
output_file

'model_C=1.0.bin'

In [23]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [24]:
!ls -lh *.bin

-rw-r--r--  1 mfilipav  staff   2.4K Dec 16 19:16 model_C=1.0.bin


In [25]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

## Load the model

In [4]:
# restart notebook and then call pickle
import pickle

In [5]:
input_file = 'model_C=1.0.bin'

In [6]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [7]:
model

LogisticRegression(max_iter=1000)

In [9]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [11]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [12]:
X = dv.transform([customer])

In [15]:
# row 0, 2nd number is probability
y_pred = model.predict_proba(X)[0, 1]

In [16]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6287446340593218


## Making requests

In [40]:
import requests

In [41]:
url = 'http://localhost:9696/predict'

In [42]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
response = requests.post(url, json=customer).json()

In [39]:
response

{'churn': True, 'churn_probability': 0.5133820686195286}

In [27]:
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d
